# Solution Architecture

![](./src/imgs/semanticSearchArchitectire.png)


* Two very important concepts in Elasticsearch are documents and indexes.

* A document is collection of fields with their associated values.

* To work with Elasticsearch you have to organize your data into documents, and then add all your documents to an index.

* Index as a collection of documents that is stored in a highly optimized format designed to perform efficient searches.

In [6]:
from typing import List
import requests
from pprint import pprint
import json
from pathlib import Path
from sentence_transformers import SentenceTransformer
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from elasticsearch import Elasticsearch
import os
from tqdm.autonotebook import tqdm
import warnings

warnings.filterwarnings("ignore")

In [5]:
with open("./data/documents.json", "rt") as file_in:
    docs_raw = json.load(file_in)
    
documents = []

for course_dict in docs_raw:
    for doc in course_dict["documents"]:
        doc["course"] = course_dict["course"]
        documents.append(doc)

In [7]:
pprint(documents[1])

{'course': 'data-engineering-zoomcamp',
 'question': 'Course - What are the prerequisites for this course?',
 'section': 'General course-related questions',
 'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites'}


## Step 2: Creation of Embeddings using Pretrained Models

In [8]:
# This is a new library compared to the previous modules. 
# Please perform "pip install sentence_transformers==2.7.0"
from sentence_transformers import SentenceTransformer

# if you get an error do the following:
# 1. Uninstall numpy 
# 2. Uninstall torch
# 3. pip install numpy==1.26.4
# 4. pip install torch
# run the above cell, it should work
model = SentenceTransformer("all-mpnet-base-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
len(model.encode(documents[0]["text"]))

768

In [11]:
#created the dense vector using the pre-trained model
operations = []
for doc in documents:
    # Transforming the title into an embedding using the model
    doc["text_vector"] = model.encode(doc["text"]).tolist()
    operations.append(doc)

## Step 3: Setup ElasticSearch connection

In [13]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch("http://localhost:9200")

pprint(es_client.info())

ObjectApiResponse({'name': '9fadbe1db950', 'cluster_name': 'docker-cluster', 'cluster_uuid': '6UfT67vtQyGhDYE7qBFB8Q', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})


## Step 4: Create Mappings and the index

* Mapping is the process of defining how a document, and the fields it contains, are stored and indexed.

* Each document is a collection of fields, which each have their own data type.

* We can compare mapping to a database schema in how it describes the fields and properties that documents hold, the datatype of each field (e.g., string, integer, or date), and how those fields should be indexed and stored

In [14]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} ,
            "text_vector": {"type": "dense_vector", "dims": 768, "index": True, "similarity": "cosine"},
        }
    }
}

In [15]:
index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

## Step 5: Add documents into index

In [16]:
for doc in operations:
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

## Step 6: Create end user query

In [17]:
search_term = "windows or mac?"
vector_search_term = model.encode(search_term)

In [18]:
query = {
    "field": "text_vector",
    "query_vector": vector_search_term,
    "k": 5,
    "num_candidates": 10000, 
}

In [19]:
res = es_client.search(index=index_name, knn=query, source=["text", "section", "question", "course"])
res["hits"]["hits"][:3]

[{'_index': 'course-questions',
  '_id': 'IcSWnJAB1kb7xJ4-MPAA',
  '_score': 0.71479183,
  '_source': {'question': 'Environment - Is the course [Windows/mac/Linux/...] friendly?',
   'course': 'data-engineering-zoomcamp',
   'section': 'General course-related questions',
   'text': 'Yes! Linux is ideal but technically it should not matter. Students last year used all 3 OSes successfully'}},
 {'_index': 'course-questions',
  '_id': 'NMSWnJAB1kb7xJ4-c_O9',
  '_score': 0.61347336,
  '_source': {'question': 'WSL instructions',
   'course': 'mlops-zoomcamp',
   'section': 'Module 1: Introduction',
   'text': 'If you wish to use WSL on your windows machine, here are the setup instructions:\nCommand: Sudo apt install wget\nGet Anaconda download address here. wget <download address>\nTurn on Docker Desktop WFree Download | AnacondaSL2\nCommand: git clone <github repository address>\nVSCODE on WSL\nJupyter: pip3 install jupyter\nAdded by Gregory Morris (gwm1980@gmail.com)\nAll in all softwares 

## Step 7: Perform Keyword search with Semantic Search (Hybrid/Advanced Search)

In [20]:
# Note: I made a minor modification to the query shown in the notebook here
# (compare to the one shown in the video)
# Included "knn" in the search query (to perform a semantic search) along with the filter  
knn_query = {
    "field": "text_vector",
    "query_vector": vector_search_term,
    "k": 5,
    "num_candidates": 10000
}

In [21]:
response = es_client.search(
    index=index_name,
    query={
        "match": {"section": "General course-related questions"},
    },
    knn=knn_query,
    size=5
)

In [22]:
response["hits"]["hits"][:2]

[{'_index': 'course-questions',
  '_id': 'IcSWnJAB1kb7xJ4-MPAA',
  '_score': 11.614713,
  '_source': {'text': 'Yes! Linux is ideal but technically it should not matter. Students last year used all 3 OSes successfully',
   'section': 'General course-related questions',
   'question': 'Environment - Is the course [Windows/mac/Linux/...] friendly?',
   'course': 'data-engineering-zoomcamp',
   'text_vector': [-0.02696547843515873,
    -0.0006261460366658866,
    -0.016629468649625778,
    0.052851490676403046,
    0.054765284061431885,
    -0.031339894980192184,
    0.029942575842142105,
    -0.04808565601706505,
    0.044675495475530624,
    0.005839452613145113,
    0.016233054921030998,
    0.012001159600913525,
    -0.03122226521372795,
    0.01660051755607128,
    -0.04886903986334801,
    -0.06496305018663406,
    0.04643421620130539,
    -0.009297749027609825,
    -0.0642528310418129,
    -0.013732693158090115,
    -0.01597621664404869,
    0.00862954556941986,
    -0.0244790166616